In [5]:
import nba_api



In [10]:
from nba_api.stats.static import players
from nba_api.stats.endpoints import playergamelog

# Get player dictionary
player_dict = players.get_players()

# Find player by name
lebron = [player for player in player_dict if player['full_name'] == 'LeBron James'][0]

# Query player's game log (for a specific season, e.g., 2021-22)
gamelog = playergamelog.PlayerGameLog(player_id=lebron['id'], season='2023')
df = gamelog.get_data_frames()[0]

print(df.head())


  SEASON_ID  Player_ID     Game_ID     GAME_DATE      MATCHUP WL  MIN  FGM  \
0     22023       2544  0022300848  FEB 28, 2024    LAL @ LAC  W   37   13   
1     22023       2544  0022300818  FEB 25, 2024    LAL @ PHX  L   37   12   
2     22023       2544  0022300813  FEB 23, 2024  LAL vs. SAS  W   34   13   
3     22023       2544  0022300776  FEB 13, 2024  LAL vs. DET  W   31    9   
4     22023       2544  0022300747  FEB 09, 2024  LAL vs. NOP  W   38    6   

   FGA  FG_PCT  ...  DREB  REB  AST  STL  BLK  TOV  PF  PTS  PLUS_MINUS  \
0   21   0.619  ...     5    6    8    0    2    3   2   34          11   
1   19   0.632  ...     6    7   12    2    1    3   3   28          -8   
2   22   0.591  ...     6    7    9    1    0    2   0   30          21   
3   15   0.600  ...     1    1    8    0    0    4   0   25           9   
4   13   0.462  ...     3    4   14    1    1    2   2   21          15   

   VIDEO_AVAILABLE  
0                1  
1                1  
2                

In [11]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import teamgamelog
import pandas as pd

def get_team_id(team_name):
    """
    Returns the team ID for a given team name.
    """
    nba_teams = teams.get_teams()
    for team in nba_teams:
        if team['full_name'].lower() == team_name.lower():
            return team['id']
    return None

def average_points_per_game(team_name, season):
    """
    Prints the average points per game for a given team and season.
    """
    team_id = get_team_id(team_name)
    if team_id is None:
        print("Team not found.")
        return
    
    game_log = teamgamelog.TeamGameLog(team_id=team_id, season=season)
    game_log_df = game_log.get_data_frames()[0]
    
    # Calculate average points
    average_points = game_log_df['PTS'].mean()
    print(f"{team_name} average points per game in {season}: {average_points:.2f}")

# Example usage
team_name = input("Enter team name (e.g., 'Los Angeles Lakers'): ")
season = input("Enter season (format YYYY-YY, e.g., '2021-22'): ")

average_points_per_game(team_name, season)


Los Angeles Lakers average points per game in 2023-24: 116.90


In [17]:
def average_points_against_team(team_name, opponent_team_name, season):
    """
    Calculates and prints the average points per game for a given team against another team in a specified season.
    """
    team_id = get_team_id(team_name)
    opponent_team_id = get_team_id(opponent_team_name)
    
    if team_id is None or opponent_team_id is None:
        print("One of the teams was not found.")
        return
    
    # Find games between the two teams
    game_finder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id, season_nullable=season)
    games_df = game_finder.get_data_frames()[0]
    
    # Ensure correct team abbreviations are used
    team_abbreviation = [team['abbreviation'] for team in teams.get_teams() if team['id'] == team_id][0]
    opponent_team_abbreviation = [team['abbreviation'] for team in teams.get_teams() if team['id'] == opponent_team_id][0]
    
    # Filter games against the opponent team
    games_against_opponent = games_df[games_df['MATCHUP'].str.contains(opponent_team_abbreviation)]
    
    # Calculate average points
    if not games_against_opponent.empty:
        average_points = games_against_opponent['PTS'].mean()
        print(f"Average points scored by {team_name} against {opponent_team_name} in {season}: {average_points:.2f}")
    else:
        print(f"No games found between {team_name} and {opponent_team_name} in {season}.")

# Example usage
team_name = "Boston Celtics"
opponent_team_name = "Los Angeles Lakers"
season = "2021-22"

average_points_against_team(team_name, opponent_team_name, season)


Average points scored by Boston Celtics against Los Angeles Lakers in 2021-22: 116.00


## Team Performance Metrics 

In [1]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import teamyearbyyearstats
import pandas as pd


In [2]:
# get Team ID
def get_team_id(team_name):
    nba_teams = teams.get_teams()
    for team in nba_teams:
        if team['full_name'].lower() == team_name.lower():
            return team['id']
    return None


In [6]:
from nba_api.stats.endpoints import leaguedashteamstats

def get_team_performance_metrics_v2(team_name, season):
    # Convert season to season_id format (e.g., "2021-22" to "2021")
    season_id = season.split('-')[0]
    
    # Fetch team stats
    team_stats = leaguedashteamstats.LeagueDashTeamStats(season=season_id, measure_type_detailed_defense='Base')
    team_stats_df = team_stats.get_data_frames()[0]

    # Find the team row
    team_row = team_stats_df[team_stats_df['TEAM_NAME'].str.lower() == team_name.lower()]

    if team_row.empty:
        print(f"No data found for {team_name} in {season}.")
        return

    # Extracting metrics
    metrics_dict = {
        'Points Per Game (PPG)': team_row['PTS'].values[0],
        'Field Goal Percentage (FG%)': team_row['FG_PCT'].values[0],
        '3-Point Percentage (3P%)': team_row['FG3_PCT'].values[0],
        'Rebounds Per Game (RPG)': team_row['REB'].values[0],
        'Assists Per Game (APG)': team_row['AST'].values[0],
        'Turnovers Per Game (TOPG)': team_row['TOV'].values[0],
        'Points Allowed Per Game (PAPG)': team_row['PTS_AGAINST'].values[0] if 'PTS_AGAINST' in team_row else 'N/A'
    }

    return metrics_dict


In [8]:
team_name = "Los Angeles Lakers"  # Example team
season = "2022-23"  # Example season
metrics = get_team_performance_metrics_v2(team_name, season)
if metrics:
    for metric, value in metrics.items():
        print(f"{metric}: {value}")


No data found for Los Angeles Lakers in 2022-23.


In [13]:
def get_team_performance_metrics_simple(season):
    # Fetching team stats for the given season
    team_stats = leaguedashteamstats.LeagueDashTeamStats(season=season)
    team_stats_df = team_stats.get_data_frames()[0]
    
    # Filter for Los Angeles Lakers
    lakers_stats = team_stats_df[team_stats_df['TEAM_NAME'] == 'Los Angeles Lakers']
    
    if lakers_stats.empty:
        print(f"No data found for Los Angeles Lakers in {season}.")
    else:
        games_played = lakers_stats['GP'].values[0]
        total_points = lakers_stats['PTS'].values[0]
        total_rebounds = lakers_stats['REB'].values[0]
        total_assists = lakers_stats['AST'].values[0]
        total_turnovers = lakers_stats['TOV'].values[0]

        # Calculate the average stats per game
        ppg = total_points / games_played if games_played else 0
        rpg = total_rebounds / games_played if games_played else 0
        apg = total_assists / games_played if games_played else 0
        topg = total_turnovers / games_played if games_played else 0

        metrics_dict = {
            'Points Per Game (PPG)': ppg,
            'Field Goal Percentage (FG%)': lakers_stats['FG_PCT'].values[0],
            '3-Point Percentage (3P%)': lakers_stats['FG3_PCT'].values[0],
            'Rebounds Per Game (RPG)': rpg,
            'Assists Per Game (APG)': apg,
            'Turnovers Per Game (TOPG)': topg,
        }
        for metric, value in metrics_dict.items():
            print(f"{metric}: {value:.2f}")

# Example usage
season = "2023-24"  # Example season
get_team_performance_metrics_simple(season)


Points Per Game (PPG): 116.90
Field Goal Percentage (FG%): 0.50
3-Point Percentage (3P%): 0.37
Rebounds Per Game (RPG): 42.52
Assists Per Game (APG): 28.47
Turnovers Per Game (TOPG): 13.75
